In [2]:
import pandas as pd

## Model Training

In [3]:
df=pd.read_csv('./data/gemstone.csv')
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [4]:
df=df.drop('id',axis=1)
df.head(2)

,carat,cut,color,clarity,depth,table,x,y,z,price
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387


In [5]:
# Independent and Dependent features
X=df.drop('price',axis=1)
Y=df['price']

In [6]:
X.head()

,carat,cut,color,clarity,depth,table,x,y,z
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77


In [7]:
Y.head()

0    13619
1    13387
2     2772
3      666
4    14453
Name: price, dtype: int64

In [8]:
# segregating numerical and categorical columns
categorical_cols=X.columns[X.dtypes=='object']
numerical_cols=X.columns[X.dtypes!='object']


In [9]:
#define the custom ranking for each ordinal variable
cut_cat=['Fair','Good','Very Good','Premium','Ideal']
color_cat=['D','E','F','G','H','I','J']
clarity_cat=['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [10]:
from sklearn.impute import SimpleImputer #Handling missing values
from sklearn.preprocessing import StandardScaler  #Feature scaling
from sklearn.preprocessing import OrdinalEncoder  #Ordinal Encoding
## pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [11]:
# numerical pipeline
num_pipeline=Pipeline(
    steps=[('imputer',SimpleImputer(strategy='median')),
           ('scaler',StandardScaler())]
)

#Categorical Pipeline
cat_pipeline=Pipeline(
    steps=[('imputer',SimpleImputer(strategy='most_frequent')),
           ('ordinalencoder',OrdinalEncoder(categories=[cut_cat,color_cat,clarity_cat])),
           ('scaler',StandardScaler())]
)

preprocessor=ColumnTransformer([
    ('num_pipeline',num_pipeline,numerical_cols),
    ('cat_pipeline',cat_pipeline,categorical_cols)
]
)

In [12]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.30,random_state=30)

In [16]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out()) #transformed
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out()) #transformed

In [17]:
X_test.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,-0.564688,-0.942132,-0.642862,-0.429765,-0.464061,-0.500036,-0.132136,-0.935071,0.019720
1,-0.175556,1.000906,-0.121531,-0.042137,-0.028595,0.036132,-1.138347,0.912774,-0.646786
2,-1.061913,0.260701,-0.121531,-1.304180,-1.298703,-1.268060,0.874076,0.912774,2.685743
3,0.970223,-0.201927,1.963794,1.048629,0.996563,0.978049,-0.132136,0.296826,0.019720
4,-0.932202,-1.312235,0.399800,-1.006699,-0.990248,-1.065186,-0.132136,-0.935071,0.686225


In [18]:
#model training
from sklearn.linear_model import LinearRegression,Ridge,Lasso,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error


In [19]:
model=LinearRegression()
model.fit(X_train,y_train)

LinearRegression()

In [20]:
model.coef_

array([ 6433.66003594,  -132.75843566,   -70.42922179, -1720.30971463,
        -499.29302619,   -63.39317848,    72.44537247,  -460.41604642,
         650.76431652])

In [21]:
model.intercept_

3970.7662895476774

In [26]:
import numpy as np
def evaluate_model(actual,predicted):
    mae=mean_absolute_error(actual,predicted)
    mse=mean_squared_error(actual,predicted)
    rmse=np.sqrt(mse)
    r2score=r2_score(actual,predicted)
    return mae,mse,rmse,r2score

In [30]:
#Train multiple model
models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'ElasticNet':ElasticNet()
    
}
trained_model_list=[]
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)
    
    #make predictions
    y_pred=model.predict(X_test)
    
    mae,mse,rmse,r2score=evaluate_model(y_test,y_pred)
    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    print('Model Training Performance')
    print('RMSE:',rmse)
    print('MAE:',mae)
    print('MSE:',mse)
    print('R2_score:',r2score*100)
    
    r2_list.append(r2score)
    print('-'*35)
    print('\n')  

LinearRegression
Model Training Performance
RMSE: 1013.9047094344004
MAE: 674.0255115796832
MSE: 1028002.7598132559
R2_score: 93.68908248567512
-----------------------------------


Lasso
Model Training Performance
RMSE: 1013.8784226767013
MAE: 675.0716923362161
MSE: 1027949.4559693959
R2_score: 93.68940971841704
-----------------------------------


Ridge
Model Training Performance
RMSE: 1013.9059272771647
MAE: 674.0555800798206
MSE: 1028005.2293677672
R2_score: 93.68906732505937
-----------------------------------


ElasticNet
Model Training Performance
RMSE: 1533.4162456064048
MAE: 1060.7368759154729
MSE: 2351365.382289642
R2_score: 85.56494831165182
-----------------------------------


